<a href="https://colab.research.google.com/github/Shikhasingh12321/DR-detect/blob/main/Image_output_ViT_and_VGG_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile
from google.colab import files
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Upload the zip file
uploaded = files.upload(r"C:\Users\Daksh\Desktop\Aptos2019_data.zip")
# Extract the zip file
import io
for file_name, file_obj in uploaded.items():
    with io.BytesIO(file_obj) as zip_buffer:
        with zipfile.ZipFile(zip_buffer) as zip_ref:
          zip_ref.extractall("/content/sample_data")

Saving Aptos2019_data.zip to C:\Users\Daksh\Desktop\Aptos2019_data.zip/Aptos2019_data.zip


In [2]:

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import timm
!pip install --upgrade torchcam
import numpy as np
import matplotlib.pyplot as plt
from torchcam.methods import CAM
from torchvision import datasets

# Define data transforms
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load test dataset
test_dataset = datasets.ImageFolder('/content/sample_data/Aptos2019/train_test', transform=transform)

# Define test loader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class LesionDetectionModel(nn.Module):
    def __init__(self):
        super(LesionDetectionModel, self).__init__()
        self.vgg19_features = torchvision.models.vgg19(weights=torchvision.models.VGG19_Weights.DEFAULT).features
        self.vgg19_fc = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 5)
        )
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=5)

    def forward(self, x):
        vgg19_output = self.vgg19_features(x)
        vgg19_output = vgg19_output.view(vgg19_output.size(0), -1)
        vgg19_output = self.vgg19_fc(vgg19_output)

        vit_output = self.vit(x)

        return vgg19_output, vit_output

model = LesionDetectionModel()

# Get a sample image
sample_image, sample_label = next(iter(test_loader))
sample_image = sample_image[0]  # Get the first image from the batch
sample_image = sample_image.unsqueeze(0)  # Add a batch dimension

# Move the sample image to the GPU if available
if torch.cuda.is_available():
    sample_image = sample_image.cuda()
    model.cuda()
    # Forward the inputs through the model
with torch.no_grad():
    _ = model(sample_image)


from torchcam.methods import CAM

# Create a CAM object
# Print the model's architecture
print(model)

# Forward the inputs through the model
outputs = model(sample_image)

# Get the CAM
sample_label = 0  # Replace with your actual label
cam_image = cam(sample_image, sample_label, outputs)

# Plot the CAM
plt.imshow(cam_image.squeeze().cpu().numpy(), cmap='hot', interpolation='nearest')
plt.show()


# Plot the original image with the CAM
plt.imshow(sample_image.squeeze().permute(1, 2, 0).cpu().numpy())
plt.imshow(cam_image.squeeze().cpu().numpy(), cmap='hot', interpolation='nearest', alpha=0.5)
plt.show()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:09<00:00, 60.7MB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

LesionDetectionModel(
  (vgg19_features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 

NameError: name 'cam' is not defined